#### data
* bytes of length n > 0

#### form
* integer: sign, endian
* string: encoding
* flag: bool vector
* byte: hex
* float

In [13]:
import time

In [16]:
def Valuer(data, form):
    result = None
    if form == "hex":
        result = data.hex()
    if form == "string":
        result = data.decode("utf-8")
    if type(form) == tuple:
        if form[0] == "integer":
            result = int.from_bytes(data, form[1])
            if len(form) == 3 and form[2] == "time":
                result = time.ctime(result)
        if form[0] == "flag":
            n = len(data)
            result = list()
            for i in range(0, n):
                byte = data[i]
                for j in range(0, 8):
                    bit = bool(byte & (1<<(j)))
                    if bit:
                        assert form[1][8*i+j] != "Reserved"
                        result.append(form[1][8*i+j])
        if form[0] == "dict":
            assert type(form[1]) == dict
            if data in form[1].keys():
                result = form[1][data]
            else:
                result = data
        if form[0] == "Fixed":
            assert data == form[1]
            result = form[2]
    return result

In [17]:
def Parser(data, form):
    index = 0
    result = list()
    for field in form:
        temp = list()
        temp.append(field[0])
        length = 0
        if type(field[1]) == int:
            length = field[1] if field[1] >= 0 else len(data) -index + field[1]
        if type(field[1]) == str:
            for item in result:
                if item[0] == field[1]:
                    length = item[2]
                    break
        if type(field[1]) == tuple:
            for item in result:
                if item[0] == field[1][0]:
                    flag = int.from_bytes(item[1], "little")
                    if flag & (1<<field[1][1]):
                        if type(field[1][2]) == bytes:
                            while data[index+length:index+length+1] != field[1][2]:
                                length += 1
                            length += 1
                            break
                        if type(field[1][2]) == int:
                            length = field[1][2]
                            break
                        if type(field[1][2]) == tuple:
                            length = int.from_bytes(data[index:index+field[1][2][0]], field[1][2][1])
                            index += field[1][2][0]
                            break
        piece = data[index:index+length]
        temp.append(piece)
        value = Valuer(piece, field[2])
        temp.append(value)
        result.append(temp)
        index += length
    assert index == len(data)
    return result

In [20]:
def Display(result):
    for field in result:
        if field[2] != str():
            tag = field[0]
            if type(field[2]) == str and len(field[2]) > 64:
                value = field[2][:64] + "..."
            else:
                value = field[2]
            print("{:16}\t{}".format(tag, value))